In [1]:
import os
os.chdir('..')

In [2]:
from matplotlib import pyplot as plt
import numpy as np
from timeit import default_timer
import pandas as pd
import seaborn as sns
from src import detector_descriptor as dd
from src import data as dt
from src import util
from src import keypoint_processing as kpp
from src import experiments as ex
import yaml
import cv2
from math import sqrt

In [3]:
image_set_variance = {
    'bark': 'zoom_rotation',
    'boat': 'zoom_rotation',
    'leuven': 'light',
    'bikes': 'blur',
    'trees': 'blur',
    'wall': 'viewpoint',
    'graf': 'viewpoint',
    'ubc': 'jpeg-compression'
}


with open(os.path.join('./', 'config.yml'), 'r') as ymlfile:
    cfg = yaml.load(ymlfile, Loader=yaml.FullLoader)
    cfg = cfg['default']
print(os.getcwd())
dataset_path = cfg['path']['dataset']
dataset = cfg['current_dataset']
# labels = dt.load_labels(os.path.join(dataset_path, dataset), '.txt')
pckl_path = cfg['path']['pckl']
pckl_name = ''.join([dataset, '_label.pckl'])
label_path = os.path.join(pckl_path, pckl_name)
# dt.dump_data(labels, dump_path)
labels = dt.load_data(label_path)
# dataset_pckl_name = cfg['dataset']['dataset_type']['oxford']['pckl_name']
data_path = os.path.join(pckl_path, ''.join([dataset, '.pckl']))

D:\Programming Projects\python projects\state-of-the-binary-descriptor


In [4]:
def get_alldet_matching_results(
        image_tuple,
        label_homography,
        descriptor_name,
        excluded_det=None,
        matcher_type=cv2.DescriptorMatcher_BRUTEFORCE,
        nn_match_ratio=0.8,
        inlier_threshold=2.5):
    if excluded_det is None:
        excluded_det = []
    alldet_inlier_ratio = dict()
    for detector_name in dd.all_detectors:
        if detector_name in excluded_det:
            continue
        alldet_inlier_ratio[detector_name] = get_matching_results(image_tuple, label_homography, detector_name,
                                                              descriptor_name, matcher_type, nn_match_ratio,
                                                              inlier_threshold)
    return alldet_inlier_ratio


def get_alldes_matching_results(
        image_tuple,
        label_homography,
        detector_name,
        excluded_des=None,
        matcher_type=cv2.DescriptorMatcher_BRUTEFORCE,
        nn_match_ratio=0.8,
        inlier_threshold=2.5):
    if excluded_des is None:
        excluded_des = []
    if detector_name is not 'AKAZE':
        excluded_des.append('AKAZE')
    alldes_inlier_ratio = dict()
    for descriptor_name in dd.get_all_descriptors():
        if descriptor_name in excluded_des:
            continue
        alldes_inlier_ratio[descriptor_name] = get_matching_results(image_tuple, label_homography, detector_name,
                                                                descriptor_name, matcher_type, nn_match_ratio,
                                                                inlier_threshold)
    return alldes_inlier_ratio


def get_matching_results(
        image_tuple,
        label_homography,
        detector_name,
        descriptor_name,
        matcher_type=cv2.DescriptorMatcher_BRUTEFORCE,
        nn_match_ratio=0.8,
        inlier_threshold=2.5):
    # print(detector_name +'-'+ descriptor_name)

    kp1, desc1 = kpp.get_desc_by_det(image_tuple[0], detector_name, descriptor_name)
    kp2, desc2 = kpp.get_desc_by_det(image_tuple[1], detector_name, descriptor_name)

    matcher = cv2.DescriptorMatcher_create(matcher_type)
    nn_matches = matcher.knnMatch(desc1, desc2, 2)

    matched1 = []
    matched2 = []
    unmatched = []
    # nn_match_ratio = 0.8  # Nearest neighbor matching ratio
    for m, n in nn_matches:
        if m.distance < nn_match_ratio * n.distance:
            matched1.append(kp1[m.queryIdx])
            matched2.append(kp2[m.trainIdx])
    for kp in kp1:
        if kp not in matched1:
            unmatched.append(kp)

    true_positive = []
    true_negative = []
    false_positive = []
    false_negative = []
    inliers1 = []
    inliers2 = []
    good_matches = []
    # inlier_threshold = 2.5  # Distance threshold to identify inliers with homography check

    for i, m in enumerate(unmatched):
        col = np.ones((3, 1), dtype=np.float64)
        col[0:2, 0] = m.pt
        col = np.dot(label_homography, col)
        col /= col[2, 0]
        if 0 <= col[0] <= image_tuple[1].shape[0] and 0 <= col[1] <= image_tuple[1].shape[1]:
            false_negative.append(m)
        else:
            false_positive.append(m)

    for i, m in enumerate(matched1):
        col = np.ones((3, 1), dtype=np.float64)
        col[0:2, 0] = m.pt
        col = np.dot(label_homography, col)
        col /= col[2, 0]
        # dist = sqrt(pow(col[0, 0] - matched2[i].pt[0], 2) + \
        #             pow(col[1, 0] - matched2[i].pt[1], 2))
        dist = np.sqrt(np.power(col[0, 0] - matched2[i].pt[0], 2) + \
                       np.power(col[1, 0] - matched2[i].pt[1], 2))

        if dist < inlier_threshold:
            good_matches.append(cv2.DMatch(len(inliers1), len(inliers2), 0))
            inliers1.append(matched1[i])
            inliers2.append(matched2[i])
            true_positive.append(m)
        else:
            true_negative.append(m)

    res_image = cv2.drawMatches(image1, inliers1, image2, inliers2, good_matches, None, flags=2)
    try:
        inlier_ratio = len(inliers1) / float(len(matched1))
    except ZeroDivisionError:
        inlier_ratio = 0
    # print(f'{descriptor_name}: {inlier_ratio}')
    # print(f'{descriptor_name} Matching Results')
    # print('*******************************')
    # print('# Keypoints 1:                        \t', len(kp1))
    # print('# Keypoints 2:                        \t', len(kp2))
    # print('# Matches:                            \t', len(matched1))
    # print('# Inliers:                            \t', len(inliers1))
    # print('# Inliers Ratio:                      \t', inlier_ratio)
    accuracy = (len(true_positive) + len(true_negative))/(len(true_positive) + len(true_negative) + len(false_positive) + len(false_negative))
    precision = len(true_positive) / (len(true_positive) + len(false_positive))
    recall =  len(true_positive) / (len(true_positive) + len(true_negative))
    # print(f'Descriptor: {descriptor_name}, TP: {len(true_positive)}, FP: {len(false_positive)}, TN: {len(true_negative)}, FN: {len(false_negative)}, Total: {len(kp1)}')
    return {'Resultant image': res_image,
            'Inlier ratio': inlier_ratio,
            'TP': true_positive,
            'FN': false_negative,
            'TN': true_negative,
            'FP': false_positive,
            'Accuracy': accuracy,
            'Precision': precision,
            'Recall': recall}

In [33]:
descriptor_code = {
    'LATCH': 'LT',
    'LUCID': 'LD',
    'AKAZE': 'AZ',
    'BRIEF': 'BF',
    'BRISK': 'BK',
    'ORB': 'OB',
    'FREAK': 'FK',
    'BOOSTDESC': 'BD'
}

detector_code = {
    'HarrisLaplace': 'HL',
    'AKAZE': 'AZ',
    'KAZE': 'KZ',
    'BRISK': 'BK',
    'FAST': 'FT',
    'AGAST': 'AG',
    'ORB': 'OB',
    'StarDetector': 'SD'
}

In [32]:
image_set_name = 'bikes'
image_set = util.get_image_set(data_path, image_set_name)
image_nums = (1, 2)
excluded_des=['KAZE', 'DAISY']
comb_accuracy_avg = dict()
comb_precision_avg = dict()
comb_recall_avg = dict()
comb_inlier_ratio_avg = dict()

for descriptor_name in dd.all_descriptors:
    comb_accuracy_avg[descriptor_code[descriptor_name]] = dict()
    comb_precision_avg[descriptor_code[descriptor_name]] = dict()
    comb_recall_avg[descriptor_code[descriptor_name]] = dict()
    comb_inlier_ratio_avg[descriptor_code[descriptor_name]] = dict()
    for detector_name in dd.all_detectors:
        if descriptor_name in excluded_des:
            continue
        if descriptor_name is 'AKAZE' and detector_name is not ('AKAZE' or 'KAZE'):
            comb_accuracy_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = 'N/A'
            comb_precision_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = 'N/A'
            comb_recall_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = 'N/A'
            comb_inlier_ratio_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = 'N/A'          
            continue
        avg_accuracy = 0
        avg_precision = 0
        avg_recall = 0
        avg_inlier_ratio = 0
        for image_num in range(2, 7):
            image1 = image_set[f'{image_set_name}_img{1}']
            image2 = image_set[f'{image_set_name}_img{image_num}']
            label_name = f'{image_set_name}_img{image_num}'
            label_homography = labels[label_name]
            matching_results = get_matching_results((image1, image2), label_homography, detector_name, descriptor_name)

            avg_accuracy += matching_results['Accuracy']
            avg_precision += matching_results['Precision']
            avg_recall += matching_results['Recall']
            avg_inlier_ratio += matching_results['Inlier ratio']
        
        comb_accuracy_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = round(avg_accuracy/5, 2)
        comb_precision_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = round(avg_precision/5, 2)
        comb_recall_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = round(avg_recall/5, 2)
        comb_inlier_ratio_avg[descriptor_code[descriptor_name]][detector_code[detector_name]] = round(avg_inlier_ratio/5, 2)
#         comb_result_avg[descriptor_name][detector_name] = {
#             'Accuracy': avg_accuracy/5,
#             'Precision': avg_precision/5,
#             'Recall': avg_recall/5,
#             'Inlier ratio': avg_inlier_ratio/5
#         }
# s, t = get_inlier_ratio((image1, image2),label_homography,detector_name,descriptor_name,matcher_type=cv2.DescriptorMatcher_BRUTEFORCE,nn_match_ratio=0.8,inlier_threshold=2.5)
# dd.print_dictionary(comb_result_avg)

In [53]:
import seaborn as sns
sns.set()
# uniform_data = np.random.rand(10, 12)

df = pd.DataFrame(comb_accuracy_avg)
# cm = sns.light_palette("green", as_cmap=True)
# color = (df.meta == 'MM').map({True: 'background-color: yellow', False: ''})
# df.style.apply(lambda s: color)
# df = df.style.background_gradient(cmap='viridis')
# df = df.style.apply(lambda x: ['background-color: yellow']*3 if x == 'N/A' else ['background-color: default']*3, axis=1)
df

,LT,LD,FR,BD,AZ,BF,BK,OB
AG,0.09,0.31,0.03,0.04,N/A,0.12,0.03,0.08
KZ,0.27,0.30,0.20,0.14,N/A,0.34,0.18,0.28
AZ,0.21,0.27,0.21,0.13,0.47,0.37,0.20,0.19
FT,0.09,0.31,0.03,0.04,N/A,0.12,0.03,0.09
BK,0.06,0.27,0.09,0.06,N/A,0.15,0.10,0.08
OB,0.07,0.20,0.18,0.08,N/A,0.13,0.22,0.10
HL,0.10,0.37,0.07,0.06,N/A,0.15,0.07,0.11
SD,0.23,0.41,0.16,0.13,N/A,0.28,0.15,0.21


In [38]:
df = pd.DataFrame(comb_precision_avg)
pd.set_option('display.max_rows', 100)
df

,LT,LD,FR,BD,AZ,BF,BK,OB
AG,0.14,0.01,0.04,0.04,N/A,0.16,0.04,0.11
KZ,0.44,0.12,0.34,0.24,N/A,0.52,0.31,0.44
AZ,0.39,0.14,0.37,0.25,0.66,0.56,0.35,0.36
FT,0.14,0.01,0.04,0.04,N/A,0.16,0.04,0.11
BK,0.11,0.04,0.15,0.06,N/A,0.21,0.17,0.12
OB,0.13,0.02,0.25,0.09,N/A,0.22,0.31,0.17
HL,0.12,0.05,0.06,0.02,N/A,0.14,0.05,0.09
SD,0.38,0.22,0.26,0.22,N/A,0.40,0.29,0.32


In [39]:
df = pd.DataFrame(comb_recall_avg)
pd.set_option('display.max_rows', 100)
df

,LT,LD,FR,BD,AZ,BF,BK,OB
AG,0.53,0.02,0.31,0.25,N/A,0.44,0.34,0.40
KZ,0.86,0.13,0.76,0.69,N/A,0.80,0.80,0.79
AZ,0.91,0.14,0.86,0.83,0.92,0.85,0.90,0.79
FT,0.54,0.02,0.33,0.27,N/A,0.44,0.35,0.39
BK,0.61,0.05,0.62,0.40,N/A,0.57,0.69,0.55
OB,0.70,0.04,0.57,0.37,N/A,0.70,0.60,0.63
HL,0.45,0.04,0.25,0.08,N/A,0.34,0.24,0.27
SD,0.81,0.14,0.68,0.67,N/A,0.65,0.79,0.64


In [40]:
df = pd.DataFrame(comb_inlier_ratio_avg)
pd.set_option('display.max_rows', 100)
df

,LT,LD,FR,BD,AZ,BF,BK,OB
AG,0.53,0.02,0.31,0.25,N/A,0.44,0.34,0.40
KZ,0.86,0.13,0.76,0.69,N/A,0.80,0.80,0.79
AZ,0.91,0.14,0.86,0.83,0.92,0.85,0.90,0.79
FT,0.54,0.02,0.33,0.27,N/A,0.44,0.35,0.39
BK,0.61,0.05,0.62,0.40,N/A,0.57,0.69,0.55
OB,0.70,0.04,0.57,0.37,N/A,0.70,0.60,0.63
HL,0.45,0.04,0.25,0.08,N/A,0.34,0.24,0.27
SD,0.81,0.14,0.68,0.67,N/A,0.65,0.79,0.64
